In [ ]:
 
import numpy as np
import pyqtgraph as pg
import pyaudio
from PyQt5 import QtCore, QtGui

FS = 16000 #Hz

D = 1
S = 2
FRAMESZ = int(FS*S/D)
AMPRNGE = int(65536/256+1)
CHUNKSZ = int(FS/50) #samples
FPS = int(FS/CHUNKSZ)

IMG = np.zeros((FRAMESZ, AMPRNGE)).astype('int8')
count = 0

class MicrophoneRecorder():
    def __init__(self, signal):
        self.signal = signal
        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(format=pyaudio.paInt16,
                            channels=1,
                            rate=FS,
                            input=True,
                            frames_per_buffer=CHUNKSZ)

    def read(self):
        data = self.stream.read(CHUNKSZ)
        y = np.fromstring(data, 'int16')
        self.signal.emit(y)

    def close(self):
        self.stream.stop_stream()
        self.stream.close()
        self.p.terminate()

class SpectrogramWidget(pg.PlotWidget):
    read_collected = QtCore.pyqtSignal(np.ndarray)
    def __init__(self):
        super(SpectrogramWidget, self).__init__()

        self.img = pg.ImageItem()
        self.addItem(self.img)

        self.img_array = IMG
        self.show()

    def update(self, chunk):
        buffer = chunk
        #print(chunk)
        chunk = np.divide(chunk, 256).astype('int8')
        chunk = np.add(chunk, int(65536/256/2))
#        chunk = np.pad(chunk, (0,D-L%D), 'constant')
        chunk = chunk.reshape((-1, D))
        chunk = np.mean(chunk, axis=1).astype('int8')

        self.img_array = np.roll(self.img_array, FRAMESZ-int(CHUNKSZ/D), 0)
        self.img_array[FRAMESZ-int(CHUNKSZ/D):FRAMESZ, :] = 255
        self.img_array[FRAMESZ-int(CHUNKSZ/D):FRAMESZ, chunk] = 0

        self.img.setImage(self.img_array)

if __name__ == '__main__':
    app = QtGui.QApplication([])
    w = SpectrogramWidget()
    w.read_collected.connect(w.update)

    mic = MicrophoneRecorder(w.read_collected)

    # time (seconds) between reads

    t = QtCore.QTimer()

    t.timeout.connect(mic.read)
    t.start(int(1000/FPS)) #QTimer takes ms


    app.exec_()
    mic.close()
 